# بسم الله الرحمن الرحیم

# محمد مهدی شفیقی - پروژه نهایی درس مباحث ویژه

# تحلیل شبکه همکاری علمی CA-HepTh

این پروژه به تحلیل ویژگی‌های ساختاری و دینامیکی شبکه هم‌نویسندگی در حوزه فیزیک انرژی بالا می‌پردازد.

In [5]:
import sys
sys.path.append('../src') # اضافه کردن پوشه src به مسیر پایتون
from data_parser import build_temporal_graphs
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import community as community_louvain
import pandas as pd

# ساخت گراف‌ها با فراخوانی تابع
data_path = '../data/cit-HepTh-abstracts'
temporal_graphs, G_final = build_temporal_graphs(data_path)

print(f"گراف نهایی با {G_final.number_of_nodes()} گره و {G_final.number_of_edges()} یال ساخته شد.")

گراف نهایی با 70476 گره و 226489 یال ساخته شد.
